In [1]:
import csv
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
class TwitterScraper:
    
    def __init__(self):
        #Browserimizi aciyoruz
        self.driver = webdriver.Chrome()
        
    def login(self):
        # Twitter giriş sayfasını açiyoruz
        self.driver.get("https://twitter.com/login")

        time.sleep(5)
        box = ".r-30o5oe.r-1niwhzg.r-17gur6a.r-1yadl64.r-deolkf.r-homxoj.r-poiln3.r-7cikom.r-1ny4l3l.r-t60dpp.r-1dz5y72.r-fdjqy7.r-13qz1uu"
        # Kullanıcı giris bilgilerinin oldugu yer
        username_input = self.driver.find_element(By.CSS_SELECTOR, box)

        # Kullanıcı adı giriş alanına tıklayip bilgileri giriyoruz
        username_input.click()
        username_input.send_keys("YOUREMAIL")
        username_input.send_keys(Keys.ENTER)
        #Twitter kullanici adimiz
        time.sleep(3)
        acc_input = self.driver.find_element(By.CSS_SELECTOR, box)
        acc_input.click()
        acc_input.send_keys("YOURUSERNAME")
        acc_input.send_keys(Keys.ENTER)
        #Sifremiz
        time.sleep(3)
        password_input = self.driver.find_element(By.CSS_SELECTOR, box)
        password_input.click()
        password_input.send_keys("YOURPASSWORD.")
        password_input.send_keys(Keys.ENTER) 
        
    #csv dosyamizi olusturuyoruz, sayfa kaynagini parcaliyoruz
    def write_csv(self, username, posts_html):
        file = open(f"{username}_tweets.csv", "w", encoding = "utf-8")
        writer = csv.writer(file)
        writer.writerow(["UDate","Tweet","Reply","Retweet","Like","View"])
        #beautifulsoup ile html.parser kullanarak sayfadan cektigimiz verileri parcaliyoruz
        soup = BeautifulSoup(posts_html, "html.parser")
        tweets = soup.find_all("article", attrs={"data-testid":"tweet"})
        #User-Data, Tweet, Reply Sayisi, Retweet Sayisi, Like Sayisi, Toplam goruntulenme sayilarini cekiyoruz.
        for bod in tweets:
            udate = bod.find("div", attrs={"class":"css-1dbjc4n r-18u37iz r-1wbh5a2 r-13hce6t"}).text
            tweet = bod.find("div", attrs={"class":"css-901oao css-cens5h r-1nao33i r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"}).text
            reply = bod.find("div", attrs={"data-testid":"reply"}).text
            retweet = bod.find("div", attrs={"data-testid":"retweet"}).text
            like = bod.find("div", attrs={"data-testid":"like"}).text
            view = bod.find("a", attrs={"class":"css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1777fci r-bt1l66 r-1ny4l3l r-bztko3 r-lrvibr"}).text
            #Cektigimiz bilgileri csv dosyamiza yaziyoruz.
            writer.writerow([udate,tweet,reply,retweet,like,view])
            
    #scroll islemi, login islemi yapiyoruz
    def get_tweets_by_username(self, username, max_posts:int=5) -> pd.DataFrame:
        #login
        self.login()
        time.sleep(5)
        # Target account
        url = f'https://twitter.com/{username}'
        self.driver.get(url)
        time.sleep(8)    
        #outerhtml ye gore sayfayi aliyoruz
        posts_html = ""
        for i in range(max_posts):
            try:
                #sayfayi article lara bolerek sayfadan cekiyorum
                post = self.driver.find_elements(By.TAG_NAME, 'article')[i]
            except Exception:
                break
            #scroll islemi yapiyoruz
            self.driver.execute_script("arguments[0].scrollIntoView(true);", post)
            time.sleep(3)
            posts_html += post.get_attribute('outerHTML')
        
        self.write_csv(username, posts_html)
        df = pd.read_csv(f"{username}_scrapper_tweets.csv")
        return df
    
    
def main():
    scraper = TwitterScraper()
    #Hedef hesabi ve cekmek isteigimiz gonderi adedini giriyoruz
    #Hedef hesabi yazarken "https://twitter.com/" dan sonraki bilgileri yaziyoruz.
    df = scraper.get_tweets_by_username("AltayCemMeric", 50)
    print(df)
    #df = pd.read_csv("AltayCemMeric_tweets.csv")
    #print(df)


if __name__ == "__main__":
    main()

                    UDate                                              Tweet  \
0   @AltayCemMeric·18 Eyl  Altay Cem Meriç adında genç bir yazarın eserle...   
1       @AltayCemMeric·8d  Anadolu suçun merkezi hacı hiç Müge Anlı izlem...   
2      @AltayCemMeric·30d  Hatırat yazarsan okuruz. Anadoluda hala peygam...   
3      @AltayCemMeric·38d  Bak buna gerçekten inanıyorlar. Baya baya inan...   
4      @AltayCemMeric·43d  Hevesinizi kırmak istemem ama Anadolu zannetme...   
5      @AltayCemMeric·46d                      Anadoludan bildiriyorum. YOK.   
6      @AltayCemMeric·52d  Kalkın MAHVOLDUK. \r\n\r\nBir 30-40 sene de bu...   
7       @AltayCemMeric·2s  Diyanet işleri başkanlığı Troy’a geçmiş. Umarı...   
8       @AltayCemMeric·3s                 Hangi belediyeler Troy’a geçiyor ?   
9    @AltayCemMeric·8 Kas                         Terörist babandır bebeğim.   
10   @AltayCemMeric·7 Kas  Boykota katılsanız kazanacağınız sempatinin ve...   
11   @AltayCemMeric·7 Kas               

In [5]:
data = pd.read_csv("AltayCemMeric_tweets.csv")

In [6]:
data

,UDate,Tweet,Reply,Retweet,Like,View
0,@AltayCemMeric·18 Eyl,Altay Cem Meriç adında genç bir yazarın eserle...,65,123,1 B,693 B
1,@AltayCemMeric·8d,Anadolu suçun merkezi hacı hiç Müge Anlı izlem...,4,1,45,1 B
2,@AltayCemMeric·30d,Hatırat yazarsan okuruz. Anadoluda hala peygam...,7,7,118,5 B
3,@AltayCemMeric·38d,Bak buna gerçekten inanıyorlar. Baya baya inan...,9,14,228,7 B
4,@AltayCemMeric·43d,Hevesinizi kırmak istemem ama Anadolu zannetme...,20,12,236,8 B
5,@AltayCemMeric·46d,Anadoludan bildiriyorum. YOK.,20,8,413,20 B
6,@AltayCemMeric·52d,Kalkın MAHVOLDUK. \r\n\r\nBir 30-40 sene de bu...,94,97,989,51 B
7,@AltayCemMeric·2s,Diyanet işleri başkanlığı Troy’a geçmiş. Umarı...,25,242,2 B,24 B
8,@AltayCemMeric·3s,Hangi belediyeler Troy’a geçiyor ?,77,103,822,28 B
9,@AltayCemMeric·8 Kas,Terörist babandır bebeğim.,51,321,3 B,89 B


In [ ]:
data = pd.read_csv("scrapper_excel_tweets.csv")